# Players dataset preparation

## Reading and initial processing

In [1]:
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = 'plotly'
pio.templates.default = "seaborn"

In [2]:
# Read datasets
df_tennis = pd.read_csv("./datasets/tennis_matches_cleaned.csv", parse_dates=["tourney_date"])
df_male = pd.read_csv("./datasets/male_players.csv")
df_female = pd.read_csv("./datasets/female_players.csv")

df_male.drop_duplicates(inplace=True)
df_female.drop_duplicates(inplace=True)

def preprocess_strings(df):
    df = df.applymap(lambda x:x.lower().strip() if type(x) == str else x)
    return df.replace(r"\s{2,}", " ", regex=True)

df_tennis.tourney_date = pd.to_datetime(df_tennis.tourney_date, format='%Y%m%d')

# Normalize strings
df_tennis = preprocess_strings(df_tennis)
df_male = preprocess_strings(df_male)
df_female = preprocess_strings(df_female)

In [3]:
# Create a new players dataset whose name is the concatenation of the name and the surname for the male and female players
df = pd.concat([df_male.name + " " + df_male.surname], axis=1, keys=["name", "gender"])
df["gender"] = "m"
df_players = df

df = pd.concat([df_female.name + " " + df_female.surname], axis=1, keys=["name", "gender"])
df["gender"] = "f"
df_players = df_players.append(df)

# Remove duplicates arising from the intersection
df_players = df_players.drop_duplicates(subset=["name"])
df_players = df_players.sort_values(by=["name"])
df_players.reset_index(drop=True, inplace=True)

# Merge the players dataset with the tennis dataset
df = pd.merge(df_tennis, df_players.rename(columns = {'name': 'winner_name', 'gender':'winner_gender'}), on="winner_name", how="left")
df = pd.merge(df, df_players.rename(columns = {'name': 'loser_name', 'gender':'loser_gender'}), on="loser_name", how="left")

# Find the players who do not have a gender and assign them the most common among the genders of the players they played with. 
winners = df[df.winner_gender.isnull()].groupby(["winner_name", "loser_gender"]).loser_gender.count().reset_index(name="occurrences").rename(columns={"winner_name":"name", "loser_gender":"gender"})

losers = df[df.loser_gender.isnull()].groupby(["loser_name", "winner_gender"]).winner_gender.count().reset_index(name="occurrences").rename(columns={"loser_name":"name", "winner_gender":"gender"})

estimate = pd.concat([winners, losers]).groupby(["name", "gender"]).occurrences.sum().reset_index(name="occurrences")
estimate = estimate.sort_values(by=["name", "occurrences"], ascending=[1, 0]).drop_duplicates("name", keep="first").drop(columns=["occurrences"])

# Create the players dataset
df_players = df.loc[:, ['winner_name', 'winner_gender']].set_axis(['name', 'gender'], axis=1, inplace=False)
df_players = df_players.append(df.loc[:, ['loser_name', 'loser_gender']].set_axis(['name', 'gender'], axis=1, inplace=False))
df_players = df_players.append(estimate)
df_players = df_players.dropna().drop_duplicates()
df_players = df_players.sort_values(by=["name"]).reset_index(drop=True)
df_players.head()

,name,gender
0,aada inna,f
1,aalisha alexis,f
2,aaliya ebrahim,f
3,aaliyah hohmann,f
4,aalyka ebrahim,f


In [4]:
df_tennis.head()

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,...,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,score_norm,games_list,w_gmsWon,l_gmsWon
0,0,2019-m020,brisbane,hard,32.0,a,2018-12-31,300.0,105453.0,NaN,...,9.0,3590.0,16.0,1977.0,3928.0,742618.69,6-4 3-6 6-2,"['6', '4', '3', '6', '6', '2']",15,12
1,1,2019-m020,brisbane,hard,32.0,a,2018-12-31,299.0,106421.0,NaN,...,16.0,1977.0,239.0,200.0,3928.0,742618.69,7-6 6-2,"['7', '6', '6', '2']",13,8
2,2,2019-m020,brisbane,hard,32.0,a,2018-12-31,298.0,105453.0,NaN,...,9.0,3590.0,40.0,1050.0,3928.0,742618.69,6-2 6-2,"['6', '2', '6', '2']",12,4
3,3,2019-m020,brisbane,hard,32.0,a,2018-12-31,297.0,104542.0,pr,...,239.0,200.0,31.0,1298.0,3928.0,742618.69,6-4 7-6,"['6', '4', '7', '6']",13,10
4,4,2019-m020,brisbane,hard,32.0,a,2018-12-31,296.0,106421.0,NaN,...,16.0,1977.0,18.0,1855.0,3928.0,742618.69,6-7 6-3 6-4,"['6', '7', '6', '3', '6', '4']",18,14


## Feature engineering

### Minutes

### Tourneys played

In [5]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id']].rename(columns={'loser_name': 'name'})

#  giocatori e relativo numero di partecipazioni ai tornei
df = pd.concat([df_w, df_l]).groupby('name').tourney_id.nunique().reset_index(name="total_tourneys_played")

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare dataframe
df

,name,total_tourneys_played
0,aada inna,1
1,aalisha alexis,2
2,aaliya ebrahim,7
3,aaliyah hohmann,2
4,aalyka ebrahim,4
...,...,...
10098,zuzana zalabska,1
10099,zuzana zlochova,88
10100,zuzanna bednarz,2
10101,zuzanna szczepanska,3


### Matches played

In [6]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id']].rename(columns={'loser_name': 'name'})

#  giocatori e relativo numero di match giocati
df = pd.concat([df_w, df_l]).groupby('name').tourney_id.count().reset_index(name="total_matches_played")

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare dataframe
df

,name,total_matches_played
0,aada inna,1
1,aalisha alexis,2
2,aaliya ebrahim,9
3,aaliyah hohmann,3
4,aalyka ebrahim,4
...,...,...
10098,zuzana zalabska,3
10099,zuzana zlochova,179
10100,zuzanna bednarz,2
10101,zuzanna szczepanska,3


### Matches won

In [7]:
df = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})

#  giocatori e relativo numero di match vinti
df = df.groupby('name').tourney_id.count().reset_index(name="total_matches_won")

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare dataframe
df

,name,total_matches_won
0,aaliya ebrahim,2
1,aaliyah hohmann,1
2,aanu ayegbusi,3
3,aaron addison,2
4,aastha dargude,1
...,...,...
6174,zulay castaneda,1
6175,zuzana luknarova,1
6176,zuzana pavelekova,2
6177,zuzana zalabska,2


### Mean, max, min won_matches_ratio in a tourney

In [8]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id']].rename(columns={'loser_name': 'name'})

matches_won_per_tourney = df_w.groupby(['name', 'tourney_id']).tourney_id.count().reset_index(name="total_matches_won_per_tourney")

matches_played_per_tourney = pd.concat([df_w, df_l]).groupby(['name', 'tourney_id']).tourney_id.count().reset_index(name="total_matches_played_per_tourney")
df = matches_played_per_tourney.merge(matches_won_per_tourney, on=['name', 'tourney_id'], how="left")
df.total_matches_won_per_tourney = df.total_matches_won_per_tourney.fillna(0).astype('Int32')
df["won_ratio"] = df.total_matches_won_per_tourney / df.total_matches_played_per_tourney

df = df.groupby(['name']).won_ratio.agg(mean_won_ratio="mean", max_won_ratio="max", min_won_ratio="min")
df_players = df_players.merge(df, on="name")
del matches_won_per_tourney, matches_played_per_tourney

### Height

Feature probabilmente inutilizzabile, l'abbiamo solo per 541 giocatori

In [9]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_ht', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_ht': 'ht'})
df_l = df_tennis.loc[:, ['loser_name', 'loser_ht', 'tourney_date']].rename(columns={'loser_name': 'name', 'loser_ht': 'ht'})

# giocatori e relativa altezza
df = pd.concat([df_w, df_l]).sort_values('tourney_date', ascending='False').groupby('name').ht.first().reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('Numero giocatori per cui abbiamo altezza ' + str(len(df[df.ht.notnull()])))
print('Numero giocatori totali ' + str(len(df)))
df

Numero giocatori per cui abbiamo altezza 541
Numero giocatori totali 10103


,name,ht
0,aada inna,NaN
1,aalisha alexis,NaN
2,aaliya ebrahim,NaN
3,aaliyah hohmann,NaN
4,aalyka ebrahim,NaN
...,...,...
10098,zuzana zalabska,NaN
10099,zuzana zlochova,NaN
10100,zuzanna bednarz,NaN
10101,zuzanna szczepanska,NaN


### Age

In [10]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_age', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_age': 'age'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_age', 'tourney_date']]).rename(columns={'loser_name': 'name', 'loser_age': 'age'})

# giocatori e relativa età
df = pd.concat([df_w, df_l]).sort_values('tourney_date', ascending='False').groupby('name').age.first().reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('Numero giocatori per cui abbiamo l\'età ' + str(len(df[df.age.notnull()])))
print('Numero giocatori totali ' + str(len(df)))
df


Numero giocatori per cui abbiamo l'età 8012
Numero giocatori totali 10103


,name,age
0,aada inna,NaN
1,aalisha alexis,NaN
2,aaliya ebrahim,NaN
3,aaliyah hohmann,NaN
4,aalyka ebrahim,NaN
...,...,...
10098,zuzana zalabska,30.647502
10099,zuzana zlochova,26.020534
10100,zuzanna bednarz,NaN
10101,zuzanna szczepanska,NaN


### Hand

In [11]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_hand']].rename(columns={'winner_name': 'name', 'winner_hand': 'hand'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_hand']]).rename(columns={'loser_name': 'name', 'loser_hand': 'hand'})

# giocatori e relativa mano preferita
df = pd.concat([df_w, df_l]).groupby('name').hand.agg(pd.Series.mode).reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('Numero giocatori per cui abbiamo la mano ' + str(len(df[df.hand.notnull()])))
print('Numero giocatori totali ' + str(len(df)))
df

Numero giocatori per cui abbiamo la mano 10103
Numero giocatori totali 10103


,name,hand
0,aada inna,u
1,aalisha alexis,u
2,aaliya ebrahim,r
3,aaliyah hohmann,u
4,aalyka ebrahim,u
...,...,...
10098,zuzana zalabska,r
10099,zuzana zlochova,r
10100,zuzanna bednarz,u
10101,zuzanna szczepanska,r


### Ranking

In [12]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_rank_points', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_rank_points': 'rank_points'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_rank_points', 'tourney_date']]).rename(columns={'loser_name': 'name', 'loser_rank_points': 'rank_points'})

# giocatori e relative misure su ranking (varianza, media, max)
df = pd.concat([df_l, df_w]).sort_values('tourney_date', ascending = False).groupby('name').rank_points.agg(mean_rank_points='mean', max_rank_points='max', last_rank_points='first', variance_rank_points=lambda x: np.var(x,ddof=0)).reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('numero di valori null mean_rank_points ' + str(df.mean_rank_points.isna().sum()) + ' su ' + str(len(df.mean_rank_points)))
print('numero di valori null max_rank_points ' + str(df.max_rank_points.isna().sum()) + ' su ' + str(len(df.mean_rank_points)))
print('numero di valori null last_rank_points ' + str(df.last_rank_points.isna().sum()) + ' su ' + str(len(df.mean_rank_points)))
print('numero di valori null variance_rank_points ' + str(df.variance_rank_points.isna().sum()) + ' su ' + str(len(df.variance_rank_points)))
df

numero di valori null mean_rank_points 5656 su 10103
numero di valori null max_rank_points 5656 su 10103
numero di valori null last_rank_points 5656 su 10103
numero di valori null variance_rank_points 5656 su 10103


,name,mean_rank_points,max_rank_points,last_rank_points,variance_rank_points
0,aada inna,NaN,NaN,NaN,NaN
1,aalisha alexis,NaN,NaN,NaN,NaN
2,aaliya ebrahim,NaN,NaN,NaN,NaN
3,aaliyah hohmann,NaN,NaN,NaN,NaN
4,aalyka ebrahim,NaN,NaN,NaN,NaN
...,...,...,...,...,...
10098,zuzana zalabska,14.000000,14.0,14.0,0.000000
10099,zuzana zlochova,87.782123,166.0,142.0,1924.036328
10100,zuzanna bednarz,NaN,NaN,NaN,NaN
10101,zuzanna szczepanska,NaN,NaN,NaN,NaN


### Spectator

In [13]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_spectators']].rename(columns = {'winner_name': 'name'})
df_l = (df_tennis.loc[:, ['loser_name', 'tourney_spectators']]).rename(columns = {'loser_name': 'name'})

# giocatori e relative misure sugli spectator (media e max)
df = pd.concat([df_l, df_w]).groupby('name').tourney_spectators.agg(mean_tourney_spectators='mean', max_tourney_spectators='max').reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('Numero di nulli in mean_tourney_spectators ' + str(df.mean_tourney_spectators.isna().sum()))
print('Numero di nulli in max_tourney_spectators ' + str(df.max_tourney_spectators.isna().sum()))
df


Numero di nulli in mean_tourney_spectators 0
Numero di nulli in max_tourney_spectators 0


,name,mean_tourney_spectators,max_tourney_spectators
0,aada inna,4059.000000,4059.0
1,aalisha alexis,3237.500000,3628.0
2,aaliya ebrahim,3632.666667,4070.0
3,aaliyah hohmann,2792.666667,2996.0
4,aalyka ebrahim,3207.750000,3943.0
...,...,...,...
10098,zuzana zalabska,1303.000000,1303.0
10099,zuzana zlochova,3236.458101,10338.0
10100,zuzanna bednarz,2206.500000,2372.0
10101,zuzanna szczepanska,2785.333333,3628.0


### Revenue

In [14]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_revenue']].rename(columns = {'winner_name': 'name'})
df_l = (df_tennis.loc[:, ['loser_name', 'tourney_revenue']]).rename(columns = {'loser_name': 'name'})

# giocatori e relative misure su revenue (media e max)
df = pd.concat([df_l, df_w]).groupby('name').tourney_revenue.agg(mean_tourney_revenue='mean', max_tourney_revenue='max').reset_index()

# aggiungere feature al data_frame dei profili dei player
df_players = df_players.merge(df, on="name")

# stampare numero di valori nulli e dataframe
print('Numero di nulli in mean_tourney_revenue ' + str(df.mean_tourney_revenue.isna().sum()))
print('Numero di nulli in max_tourney_revenue ' + str(df.max_tourney_revenue.isna().sum()))
df

Numero di nulli in mean_tourney_revenue 0
Numero di nulli in max_tourney_revenue 0


,name,mean_tourney_revenue,max_tourney_revenue
0,aada inna,939835.240000,939835.24
1,aalisha alexis,704802.720000,717933.37
2,aaliya ebrahim,737944.000000,885863.35
3,aaliyah hohmann,551917.330000,593484.95
4,aalyka ebrahim,713766.042500,885863.35
...,...,...,...
10098,zuzana zalabska,239390.330000,239390.33
10099,zuzana zlochova,650492.769274,2291040.00
10100,zuzanna bednarz,419291.125000,454394.49
10101,zuzanna szczepanska,569497.023333,639257.36


## Feature analysis

In [15]:
df_players.describe()

,total_tourneys_played,total_matches_played,total_matches_won,mean_won_ratio,max_won_ratio,min_won_ratio,ht,age,mean_rank_points,max_rank_points,last_rank_points,variance_rank_points,mean_tourney_spectators,max_tourney_spectators,mean_tourney_revenue,max_tourney_revenue
count,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,6179.000000,505.000000,5250.000000,3826.000000,3826.000000,3826.000000,3.826000e+03,6179.000000,6179.000000,6.179000e+03,6.179000e+03
mean,29.434213,58.852403,30.062955,0.306561,0.726224,0.052722,181.283168,20.887511,163.657548,314.335860,194.495034,5.972712e+04,3473.790618,6413.273669,6.950966e+05,1.487564e+06
std,37.293222,81.333878,46.458837,0.149703,0.196863,0.169440,8.934340,4.699697,474.616229,911.911784,627.980235,5.894063e+05,1009.885439,4652.622346,2.121458e+05,1.313040e+06
min,1.000000,1.000000,1.000000,0.013889,0.333333,0.000000,155.000000,14.006845,1.000000,1.000000,1.000000,0.000000e+00,118.666667,158.000000,2.355808e+04,3.076084e+04
25%,4.000000,6.000000,2.000000,0.200000,0.500000,0.000000,175.000000,17.300479,9.000000,13.000000,8.000000,4.892231e+00,3059.886508,3834.000000,6.057221e+05,7.938150e+05
50%,12.000000,19.000000,7.000000,0.294093,0.666667,0.000000,183.000000,19.753593,31.128398,52.000000,30.000000,1.477467e+02,3255.500000,4422.000000,6.529350e+05,9.628369e+05
75%,40.000000,75.500000,36.000000,0.392857,1.000000,0.000000,188.000000,23.624914,122.307604,219.000000,116.000000,2.461451e+03,3623.365033,6305.000000,7.324217e+05,1.378907e+06
max,175.000000,387.000000,289.000000,1.000000,1.000000,1.000000,211.000000,53.642710,10235.315634,16950.000000,12113.000000,2.061166e+07,15660.000000,18086.000000,5.002794e+06,5.002794e+06


### Tourneys played

In [16]:
df_players.total_tourneys_played.hist()

In [17]:
np.log2(df_players.total_tourneys_played).hist()

### Matches played

In [18]:
df_players.total_matches_played.hist()

In [19]:
np.log(df_players.total_matches_played).hist()

### Matches won

In [20]:
df_players.total_matches_won.hist()

In [21]:
np.log10(df_players.total_matches_won).hist()

### Age

In [22]:
df_players.age.hist()

In [23]:
np.log(df_players.age).hist()

### Hand

### Ranking

In [24]:
df_players.variance_rank_points.hist()

In [25]:
np.log1p(df_players.variance_rank_points).hist()

In [26]:
df_players.last_rank_points.hist()

In [27]:
np.sqrt(np.log(df_players.last_rank_points)).hist()

## Correlation analysis

In [28]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

correlation = df_players.corr(method="pearson") #.abs()
correlation = correlation.where(np.tril(np.ones(correlation.shape)).astype(bool))
px.imshow(correlation[(correlation>=0.8) | (correlation<=-0.8)], zmin=0, zmax=1, width=1000, height=1000).show()

correlation = correlation.unstack().drop(labels=get_redundant_pairs(correlation)).sort_values(ascending=False).drop_duplicates()
print(f"These are {len(correlation[(correlation>=0.8) | (correlation<=-0.8)])} pairs whose correlation is higer than 80%:")
correlation[(correlation>=0.8) | (correlation<=-0.8)]

These are 8 pairs whose correlation is higer than 80%:


total_matches_played     total_matches_won       0.987105
total_tourneys_played    total_matches_played    0.984930
max_tourney_spectators   max_tourney_revenue     0.967092
mean_rank_points         max_rank_points         0.962243
mean_tourney_spectators  mean_tourney_revenue    0.950411
total_tourneys_played    total_matches_won       0.945272
max_rank_points          last_rank_points        0.878969
mean_rank_points         last_rank_points        0.801598
dtype: float64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56da3ab5-e195-41aa-a609-f5fefeb3379d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>